In [18]:
from pysat.formula import CNF

# parse attributes file
def parse_attr(text):
    lines = text.strip().split('\n')
    parsed = {}
    for (i, line) in enumerate(lines, start=1):
        category, items_str = line.split(':')
        items = items_str.split(',')
        parsed[category.strip()] = {items[0].strip(): i, items[1].strip(): -i} # the name of the attribute value and its numeric representation
    return parsed

def parse_constraints_into_cnf(text, attrs: dict):
    lines = text.strip().split('\n')
    cnf_repr = []
    print(lines)
    for (lineNum, line) in enumerate(lines):
        cnf_constraint = []
        clauses: list[str] = line.strip().split('AND') # A list of disjunctive clauses, e.g. A OR B AND C OR D would produce ['A OR B', 'C OR D']
        for clause in clauses:
            cnf_clause = []
            literals: list[str] = clause.strip().split('OR')
            for literal in literals:
                
                literal_name = literal.strip().removeprefix('NOT').strip()
                literal_numeric = None
                for attribute in attrs.values():
                    print(attribute)
                    if literal_name.strip() in attribute:
                        literal_numeric = attribute[literal_name]
                if (literal_numeric is None): 
                    print("Error parsing '" + literal + "' on line " + str(lineNum) )
                    return []
                if (literal_numeric is not None and literal.strip().startswith('NOT')):
                    literal_numeric = -literal_numeric

                cnf_clause.append(literal_numeric)

            cnf_constraint.append(cnf_clause)

        cnf_repr.append(cnf_constraint)
    return cnf_repr

with open("ExampleCase/attributes.txt") as attrfile:
    attr = parse_attr(attrfile.read())
    print(attr)
    with open("ExampleCase/constraints.txt") as constrfile:
        print(parse_constraints_into_cnf(constrfile.read(), attr))


{'dissert': {'cake': 1, 'ice-cream': -1}, 'drink': {'wine': 2, 'beer': -2}, 'main': {'fish': 3, 'beef': -3}}
['NOT wine OR NOT ice-cream AND beer OR ice-cream']
{'cake': 1, 'ice-cream': -1}
{'wine': 2, 'beer': -2}
{'fish': 3, 'beef': -3}
{'cake': 1, 'ice-cream': -1}
{'wine': 2, 'beer': -2}
{'fish': 3, 'beef': -3}
{'cake': 1, 'ice-cream': -1}
{'wine': 2, 'beer': -2}
{'fish': 3, 'beef': -3}
{'cake': 1, 'ice-cream': -1}
{'wine': 2, 'beer': -2}
{'fish': 3, 'beef': -3}
[[[-2, 1], [-2, -1]]]
